<a href="https://colab.research.google.com/github/guilhermelaviola/BIArchitectureAndBigData/blob/main/Class04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing all the necessary libraries:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import spacy

In [ ]:
# Importing and displaying the dataset:
df = pd.read_csv('amostra_avaliacao_loja.csv')
df

In [ ]:
# Displaying the dataset columns:
df.columns

In [ ]:
df['Texto da Avaliação'] = df['Texto da Avaliação'].fillna('').astype(str)
concat_text = ' '.join(df['Texto da Avaliação'])
words = concat_text.lower().split()
frequency = Counter(words)
df_frequency = pd.DataFrame(frequency.items(), columns=['palavra', 'frequencia'])
df_frequency.sort_values(by='frequencia', ascending=False, inplace=True)
df_frequency.reset_index(drop=True, inplace=True)
df_frequency

In [ ]:
nlp = spacy.load('pt_core_news_sm')

def find_adjectives(text):
  doc = nlp(text)
  adjectives = [token.text.lower() for token in doc if token.pos_ == 'ADJ']
  return adjectives

In [ ]:
all_adjectives = []

for evaluation in df['Texto da Avaliação'].dropna():
  all_adjectives.extend(find_adjectives(evaluation))
adjectives_frequency = Counter(all_adjectives)
df_adjectives = pd.DataFrame(adjectives_frequency.items(), columns=['Adjetivo', 'Frequencia'])
df_adjectives.sort_values(by='Frequencia', ascending=False, inplace=True)
df_adjectives.reset_index(drop=True, inplace=True)
df_adjectives

In [ ]:
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(df_adjectives.set_index('Adjetivo')['Frequencia'])

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()